# 🧠 Análisis Automatizado de Trazas de Juego
Este notebook analiza archivos JSON con trazas de eventos generadas por Amon-RA. Simplemente coloca el archivo `.json` en el mismo directorio y modifica el nombre del archivo en la celda correspondiente.

In [2]:
# Nombre del archivo JSON
file_name = 'guardado - copia.json'

In [3]:
import pandas as pd
import json

# Cargar archivo JSON línea por línea
def cargar_trazas(file_name):
    with open(file_name, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        return pd.DataFrame([json.loads(line) for line in lines])

# Cargar DataFrame
df = cargar_trazas(file_name)


## 🔢 Conteo de eventos y métricas generales

In [5]:
# Conteo de eventos principales
conteos = df['event_type'].value_counts()
conteos

event_type
Jump             282
LevelProgress    100
Death             39
FallPlatform      12
Pause              3
LevelEnd           2
BlueSlime          2
SessionStart       1
Name: count, dtype: int64

## 🦘 Análisis de Saltos

In [7]:
saltos = df[df['event_type'] == 'Jump']
print(f"Número total de saltos: {len(saltos)}")
if len(saltos) < 10:
    print('Pocos saltos: puede indicar falta de interacciones.')
elif len(saltos) > 200:
    print('Muchos saltos: jugador activo o nivel largo.')
else:
    print('Cantidad moderada de saltos: ritmo de juego equilibrado.')

Número total de saltos: 282
Muchos saltos: jugador activo o nivel largo.


## 💀 Análisis de Muertes

In [9]:
muertes = df[df['event_type'] == 'Death']
print(f"Total de muertes: {len(muertes)}")
print(muertes['death_type'].value_counts())
if len(muertes) == 0:
    print('¡No hubo muertes! El nivel puede ser demasiado fácil.')
elif muertes['death_type'].value_counts().max() > 10:
    print('Hay una causa de muerte predominante que conviene revisar.')
else:
    print('Distribución equilibrada de causas de muerte.')

Total de muertes: 39
death_type
spike apears death    18
fall death            14
spike death            7
Name: count, dtype: int64
Hay una causa de muerte predominante que conviene revisar.


## 🧟 Análisis de colisiones con BlueSlime

In [11]:
slimes = df[df['event_type'] == 'BlueSlime']
print(f"Colisiones con BlueSlime: {len(slimes)}")
if len(slimes) > 5:
    print('El jugador ha interactuado repetidamente con enemigos. Posiblemente aprendiendo o probando.')
elif len(slimes) == 0:
    print('No se ha topado con enemigos, o los ha evitado bien.')
else:
    print('Pocas colisiones: el jugador ha tenido cuidado o el diseño lo facilita.')

Colisiones con BlueSlime: 2
Pocas colisiones: el jugador ha tenido cuidado o el diseño lo facilita.


## 🕓 Análisis del tiempo por nivel

In [13]:
# Filtrar eventos LevelEnd
level_end = df[df["event_type"] == "LevelEnd"].copy()

# Asegurar que la columna 'time' esté en formato correcto: cambiar ',' por '.'
level_end["tiempo_nivel_segundos"] = (
    level_end["time"]
    .str.replace(",", ".", regex=False)
    .astype(float)
)

# Seleccionar y mostrar las columnas relevantes
nivel_duracion = level_end[["level_id", "tiempo_nivel_segundos"]].dropna().reset_index(drop=True)
nivel_duracion


,level_id,tiempo_nivel_segundos
0,b2561eb3-a0b5-410b-912c-a195b79f4c51,65.47766
1,b2561eb3-a0b5-410b-912c-a195b79f4c51,45.27765


In [14]:
if not nivel_duracion['tiempo_nivel_segundos'].dropna().empty:
    promedio = nivel_duracion['tiempo_nivel_segundos'].mean()
    print(f"Duración media por nivel: {promedio:.2f} segundos")
    if promedio < 10:
        print('🟢 Los niveles se están completando muy rápido.')
    elif promedio > 60:
        print('🔴 Los niveles tardan demasiado en completarse.')
    else:
        print('🟡 Tiempo adecuado para mantener atención y reto.')
else:
    print("No hay datos suficientes para calcular la duración promedio por nivel.")


Duración media por nivel: 55.38 segundos
🟡 Tiempo adecuado para mantener atención y reto.


## ⏱️ Tiempo total de juego (desde primer evento hasta el último)

In [16]:
df['event_timestamp'] = pd.to_datetime(df['event_timestamp'], errors='coerce')
total_tiempo = (df['event_timestamp'].max() - df['event_timestamp'].min()).total_seconds()
print(f"Tiempo total jugado: {total_tiempo:.2f} segundos")
if total_tiempo < 30:
    print('Duración muy corta. Puede que el jugador se haya salido pronto.')
elif total_tiempo > 300:
    print('Sesión larga: puede ser una prueba de nivel completa o de ambos niveles.')
else:
    print('Duración de sesión razonable.')

Tiempo total jugado: 515.87 segundos
Sesión larga: puede ser una prueba de nivel completa. O de ambos niveles.


C:\Users\Sheila Julvez\AppData\Local\Temp\ipykernel_17164\657818357.py:1: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df['event_timestamp'] = pd.to_datetime(df['event_timestamp'], errors='coerce')
